# modeling14-4-FiD-encoder-sentence-level-classifier-loading-mean-pooling-embedding
- modeling sentence-classifier
- sentence-bert

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'

In [ ]:
from pprint import pprint
from util import utils

In [ ]:
from functools import partial
import json
import math
import os
import logging
import sys
import evaluate
from util import utils
from tqdm.auto import tqdm
import pickle

import transformers
import torch
import numpy as np
import random
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoModelForSequenceClassification, 
    AutoModel, 
    AutoConfig, 
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    set_seed,
    get_scheduler,
)
from util.arguments import ModelArguments, DataTrainingArguments 
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn.functional as F
from FiD.src.model import FiDT5
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from src.data import BinaryCustomDatasetShuffle

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [ ]:
print(device)

## Get FiD Encoder Embedding from pickle file

In [ ]:
testing_file = '/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/sequence_exclude_no_answer_exclude_indecisve/testing-sequence_exclude_no_answer_exclude_indecisve_ctx100id_split_train_1.pickle'
with open(testing_file, 'rb') as f:
    test_input = pickle.load(f)


In [ ]:
type(test_input)

In [ ]:
print(test_input[0]['input_embedding'].shape)
print(test_input[0]['em_pattern'].shape)

In [ ]:
class EncoderSentenceClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, instances, shuffle=True):
        if shuffle:
            random.shuffle(instances)
        self.instances = instances

    def __len__(self):
        return len(self.instances)

    def __getitem__(self, idx):
        input_embedding_ = self.instances[idx]['input_embedding']
        em_pattern_ = self.instances[idx]['em_pattern']
        
        result = {
            'input_embedding': input_embedding_,
            'em_pattern': em_pattern_
        }

        return result

In [ ]:
test_dataset = EncoderSentenceClassificationDataset(test_input)

In [ ]:
test_dataset[0]

In [ ]:
def custom_collate(batch, padding):
    train_lst = [b['input_embedding'] for b in batch]
    label_lst = [b['em_pattern'] for b in batch]
    seq_len_lst = [b['em_pattern'].shape[0] for b in batch]
    max_seq_len = max(seq_len_lst)

    padding_train_lst = []
    for embedding in train_lst:
        if embedding.shape[0] < max_seq_len:
            post_pad = torch.full(size=(max_seq_len - embedding.shape[0], embedding.shape[1]), fill_value=padding)
            # post_pad = torch.full(size=(max_seq_len - embedding.shape[0], embedding.shape[1]), fill_value=-100)
            padding_train_lst.append(torch.concat([embedding, post_pad]))
        else:
            padding_train_lst.append(embedding)

    inputs = torch.stack(padding_train_lst)

    padding_label_lst = []
    for label in label_lst:
        if label.shape[0] < max_seq_len:
            post_pad = torch.full(size=(max_seq_len - label.shape[0],), fill_value=padding)
            # post_pad = torch.full(size=(max_seq_len - label.shape[0],), fill_value=-100)
            torch.concat([label, post_pad])
            padding_label_lst.append(torch.concat([label, post_pad]))
        else:
            padding_label_lst.append(label)

    labels = torch.stack(padding_label_lst)

    return {
        'inputs': inputs,
        'labels': labels,
        'sequence_len': torch.tensor(seq_len_lst)
    }

In [ ]:
test_dataloader = DataLoader(test_dataset,
                                  shuffle=False,
                                  collate_fn=partial(custom_collate, padding=-100),
                                  batch_size=8,
                                  )

In [ ]:
test_iter = iter(test_dataloader)

In [ ]:
cur_batch = next(test_iter)

In [ ]:
cur_batch.keys()

In [ ]:
cur_batch['sequence_len']

In [ ]:
cur_batch['inputs'].shape

In [ ]:
cur_batch['labels'].shape

In [ ]:
cur_batch['labels']

In [ ]:
cur_batch['inputs'][0][-5]